In [1]:
import os
from flask import Flask, jsonify
from supabase import create_client, Client

app = Flask(__name__)

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

TABLE_NAME = "v2_federato_amplitude_data"

In [ ]:
def get_churn_events(user_id):
    """Calculate the top 5 churn events (least likely to return to) for a given user."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Convert events to a list of event types
        events = [event["event_type"] for event in events_response.data]
        
        # Calculate return frequency for each event type
        event_returns = {}
        for i, event in enumerate(events[:-1]):  # Exclude last event
            # Look at the next events to see if user returns to this event
            future_events = events[i+1:]
            if event in future_events:
                event_returns[event] = event_returns.get(event, 0) + 1
        
        # Calculate return rate (returns / total occurrences)
        event_counts = {}
        for event in events:
            event_counts[event] = event_counts.get(event, 0) + 1
            
        # Calculate return rates, including events with 0 returns
        return_rates = {}
        for event, count in event_counts.items():
            returns = event_returns.get(event, 0)  # Get 0 if event never returned to
            return_rates[event] = returns / count
        
        # Get top 5 events by LOWEST return rate (minimum 2 occurrences)
        churn_events = sorted(
            [(event, rate) for event, rate in return_rates.items() 
             if event_counts[event] >= 2],  # Filter events with at least 2 occurrences
            key=lambda x: x[1]  # Sort ascending instead of descending
        )[:5]
        
        # Format results
        return [{"event": event, "return_rate": rate} for event, rate in churn_events]
        
    except Exception as e:
        print(f"Error calculating churn events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top churn events
def update_all_users_churn_events():
    try:
        page_size = 1000
        last_user_id = 0
        
        while True:
            # Get next batch of users
            users_response = supabase.table("user_table")\
                .select("user_id")\
                .gt("user_id", last_user_id)\
                .order("user_id")\
                .limit(page_size)\
                .execute()
            
            # If no more users, break
            if not users_response.data:
                break
            
            # Process each user in this batch
            for user in users_response.data:
                user_id = user["user_id"]
                churn_events = get_churn_events(user_id)
                
                # Update will create the column if it doesn't exist
                supabase.table("user_table")\
                    .update({"top_churn_events": churn_events})\
                    .eq("user_id", user_id)\
                    .execute()
                
                print(f"Updated churn events for user {user_id}")
                last_user_id = user_id
            
            print(f"Completed batch up to user {last_user_id}")
        
        print("Completed updating all users' churn events")
        
    except Exception as e:
        print(f"Error updating churn events: {e}")

# Execute the update
update_all_users_churn_events()


Updated churn events for user 1
Updated churn events for user 2
Updated churn events for user 3
Updated churn events for user 4
Updated churn events for user 5
Updated churn events for user 6
Updated churn events for user 7
Updated churn events for user 8
Updated churn events for user 9
Updated churn events for user 10
Updated churn events for user 11
Updated churn events for user 12
Updated churn events for user 13
Updated churn events for user 14
Updated churn events for user 15
Updated churn events for user 16
Updated churn events for user 17
Updated churn events for user 18
Updated churn events for user 19
Updated churn events for user 20
Updated churn events for user 21
Updated churn events for user 22
Updated churn events for user 23
Updated churn events for user 24
Updated churn events for user 25
Updated churn events for user 26
Updated churn events for user 27
Updated churn events for user 28
Updated churn events for user 29
Updated churn events for user 30
Updated churn event